In [1]:
import io
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
embedding_layer = tf.keras.layers.Embedding(1000,5)
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.00066633, -0.04647933,  0.02341236, -0.02410635, -0.04101088],
       [-0.02618973, -0.04920033, -0.02461977, -0.01883521, -0.00648056],
       [ 0.01043339, -0.00205213,  0.03492762, -0.01219684, -0.02905241]],
      dtype=float32)

In [3]:
(train_data,test_data),info = tfds.load(
    "imdb_reviews",
    split = ["train","test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

In [9]:
train_example_batch , train_label_batch = next(iter(train_data.batch(5)))
train_example_batch

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [10]:
# Transfer learning to convert words to vec
import tensorflow_hub as hub

In [11]:
pretained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretained_model, input_shape=[], dtype = tf.string, trainable=True)

In [12]:
hub_layer(train_example_batch[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

In [13]:
train_example_batch[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [23]:
model = tf.keras.Sequential([
    hub_layer,
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_7 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_data.shuffle(10000).batch(512), epochs=20)

Epoch 1/20
49/49 [==============================] - 2s 28ms/step - loss: 1.4757 - accuracy: 0.5169
Epoch 2/20
49/49 [==============================] - 1s 27ms/step - loss: 0.6748 - accuracy: 0.5948
Epoch 3/20
49/49 [==============================] - 1s 27ms/step - loss: 0.6456 - accuracy: 0.6300
Epoch 4/20
49/49 [==============================] - 1s 27ms/step - loss: 0.6198 - accuracy: 0.6636
Epoch 5/20
49/49 [==============================] - 1s 28ms/step - loss: 0.5944 - accuracy: 0.6978
Epoch 6/20
49/49 [==============================] - 1s 28ms/step - loss: 0.5702 - accuracy: 0.7204
Epoch 7/20
49/49 [==============================] - 1s 27ms/step - loss: 0.5462 - accuracy: 0.7454
Epoch 8/20
49/49 [==============================] - 1s 27ms/step - loss: 0.5220 - accuracy: 0.7675
Epoch 9/20
49/49 [==============================] - 1s 27ms/step - loss: 0.4998 - accuracy: 0.7890
Epoch 10/20
49/49 [==============================] - 1s 28ms/step - loss: 0.4774 - accuracy: 0.8033
Epoch 11/

In [28]:
model.evaluate(train_data.batch(512))

49/49 [==============================] - 1s 22ms/step - loss: 0.3069 - accuracy: 0.8896


[0.30685240030288696, 0.8895999789237976]

In [29]:
# model.save('Movie Review model.h5')

In [39]:
model.predict(['This is ok movie'])

array([[0.5107626]], dtype=float32)